In [38]:
import pandas as pd
import xlwings as xw

In [39]:
def get_data_with_xlwings(file_name, sheet_name):
    wb = xw.Book(file_name)
    sheet = wb.sheets[sheet_name]
    data = sheet.range('A1').options(pd.DataFrame, expand='table').value
    wb.close()
    return data

In [40]:
def get_pricing_matrix_path(service_type):
    # Dictionary mapping service types to their respective spreadsheet filenames
    service_type_to_file = {
        "FedEx First Overnight®": "path/to/ServiceType1.xlsx",
        "FedEx Priority Overnight®": "path/to/ServiceType2.xlsx",
        "FedEx Standard Overnight®": "path/to/ServiceType2.xlsx",
        "FedEx 2Day® A.M.": "path/to/ServiceType2.xlsx",
        "FedEx 2Day®": "path/to/ServiceType2.xlsx",
        "FedEx Express Saver®": "path/to/ServiceType2.xlsx",
        "Intra-Hawaii Standard List Rates": "path/to/ServiceType2.xlsx",
#         BEGIN MULTIWEIGHT PRICING MATRIX SECTION
        "FedEx First Overnight®": "path/to/ServiceType2.xlsx",
        "FedEx Priority Overnight®": "path/to/ServiceType2.xlsx",
        "FedEx Standard Overnight®": "path/to/ServiceType2.xlsx",
        "FedEx 2Day® A.M.": "path/to/ServiceType2.xlsx",
        "FedEx 2Day®": "path/to/ServiceType2.xlsx",
        "FedEx Express Saver®": "path/to/ServiceType2.xlsx",
        "Intra-Hawaii FedEx Priority Overnight®": "path/to/ServiceType2.xlsx",
#         END MULTIWEIGHT PRICING MATRIX SECTION
        "FedEx First Overnight® Freight": "path/to/ServiceType2.xlsx",
        "FedEx 1Day® Freight": "path/to/ServiceType2.xlsx",
        "FedEx 2Day® Freight": "path/to/ServiceType2.xlsx",
        "FedEx 3Day® Freight": "path/to/ServiceType2.xlsx",
    }

    if service_type in service_type_to_file:
        file_name = service_type_to_file[service_type]
        return file_name
#         return pd.read_excel(file_name)
    else:
        # Handle cases where the service type does not have a corresponding file
        return None

In [41]:
def process_data(data):
#     abstract service type/weight/zone
#     open correct pricing matrix and locate correct value
#     write value to OTPUB cell in charges sheet
#     print(data)
    for index, row in data.iterrows():
#         INDEX WOULD BE THE INVOICE NUMBER IN FREIGHT & ACCESSORIALS BOOK
        print("ROW")
        print(row["OTPUB"])
#         print("inside process_data for loop")
#         service_type = row['Service Type']
#         weight = row['RW']
#         zone = row['Z']
#         path_to_pricing_matrix = get_pricing_matrix_path(service_type) 
#         updated_otpub_value = open correct sheet to calc. what a package this weight going to this zone would be
#         update the value in the dataframe
#     print(data)
    return data

In [42]:
def save_results_with_xlwings(data, file_name, sheet_name):
    print(data)
#     wb = xw.Book(file_name)
#     ws = wb.sheets[sheet_name]
#     ws.range('A1').options(index=False).value = data
#     wb.save()
#     wb.close()

In [44]:
#BEGIN MAIN SCRIPT USING ABOVE FUNCTIONS
file_name = './charges.xlsm'
spreadsheet_name = 'FREIGHT & ACCESSORIALS'
sheet_data = get_data_with_xlwings(file_name, spreadsheet_name)
processed_data = process_data(sheet_data)
# print(processed_data['OTPUB'])
# save_results_with_xlwings(processed_data, file_name, spreadsheet_name)

ROW
'FedEx Standard Overnight'
ROW
34.14
ROW
34.14
ROW
34.14
ROW
41.17
ROW
34.14
ROW
34.59
ROW
34.59
ROW
28.04
ROW
49.06
ROW
48.41
ROW
41.17
ROW
28.04
ROW
47.59
ROW
41.17
ROW
58.99
ROW
34.59
ROW
57.6
ROW
47.59
ROW
34.14
ROW
34.14
ROW
41.17
ROW
41.17
ROW
41.17
ROW
34.14
ROW
68.98
ROW
28.04
ROW
4.86
ROW
5.72
ROW
4.58
ROW
5.44
ROW
5.72
ROW
5.44
ROW
6.29
ROW
8.01
ROW
5.44
ROW
7.72
ROW
8.87
ROW
7.15
ROW
3.15
ROW
8.87
ROW
8.58
ROW
5.72
ROW
5.44
ROW
5.44
ROW
5.72
ROW
8.3
ROW
48.41
ROW
34.04
ROW
34.59
ROW
37.74
ROW
34.59
ROW
110.72
ROW
57.6
ROW
67.29
ROW
41.52
ROW
28.04
ROW
34.59
ROW
34.04
ROW
34.04
ROW
34.04
ROW
28.04
ROW
34.04
ROW
34.04
ROW
47.85
ROW
28.04
ROW
45.54
ROW
34.59
ROW
28.04
ROW
50.84
ROW
50.84
ROW
34.04
ROW
34.04
ROW
46.91
ROW
56.39
ROW
50.55
ROW
34.04
ROW
43.82
ROW
34.04
ROW
56.39
ROW
56.39
ROW
50.55
ROW
33.37
ROW
33.37
ROW
33.37
ROW
33.37
ROW
33.37
ROW
43.82
ROW
34.04
ROW
67.29
ROW
41.52
ROW
101.28
ROW
101.28
ROW
34.04
ROW
50.84
ROW
28.04
ROW
46.91
ROW
34.04
ROW
33.37
ROW
33.37